# Quantus + NLP
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/understandable-machine-intelligence-lab/Quantus/main?labpath=tutorials%2FTutorial_NLP_Demonstration.ipynb)


This tutorial demonstrates how to use the library for evaluation explanation of text classification models.
For this purpose, we use a pre-trained `Distilbert` model from [Huggingface](https://huggingface.co/models) and `GLUE/SST2` dataset [here](https://huggingface.co/datasets/sst2).

Author: Artem Sereda

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1eWK9ebfMUVRG4mrOAQvXdJ452SMLfffv?usp=sharing)

In [1]:
import numpy as np
import pandas as pd
from datasets import load_dataset
import tensorflow as tf
import logging
from IPython.core.display import HTML
import random
#import matplotx
import matplotlib.pyplot as plt
import gc
from quantus.helpers.plotting import plot_model_parameter_randomisation_experiment
from quantus.nlp.helpers.utils import map_explanations
import quantus.nlp as qn


#plt.style.use(matplotx.styles.dracula)
logging.getLogger("absl").setLevel(logging.WARNING)
random.seed(42)
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## 1) Preliminaries

### 1.1 Load pre-trained model and tokenizer from [huggingface](https://huggingface.co/models) hub

In [2]:
model = qn.TensorFlowHuggingFaceTextClassifier.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-03-01 20:24:28.595755: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-01 20:24:28.596209: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


### 1.2 Load test split of [GLUE/SST2](https://huggingface.co/datasets/sst2) dataset

In [4]:
BATCH_SIZE = 32

dataset = load_dataset("sst2")["test"]
x_batch = dataset["sentence"][:BATCH_SIZE]
random.shuffle(x_batch)

Found cached dataset sst2 (/Users/artemsereda/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)


  0%|          | 0/3 [00:00<?, ?it/s]

Run an example inference, and demonstrate models predictions.

In [5]:
def decode_labels(y_batch: np.ndarray):
    """A helper function to map integer labels to human-readable class names."""
    return [model.internal_model.config.id2label[i] for i in y_batch]


y_batch = model.predict(x_batch).argmax(axis=-1)

# Show the x, y data.
pd.DataFrame([x_batch[:10], decode_labels(y_batch[:10])]).T

2023-03-01 20:24:55.136895: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-03-01 20:24:56.473612: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


,0,1
0,this is junk food cinema at its greasiest .,NEGATIVE
1,uneasy mishmash of styles and genres .,NEGATIVE
2,it 's just incredibly dull .,NEGATIVE
3,a feel-good picture in the best sense of the t...,POSITIVE
4,here 's a british flick gleefully unconcerned ...,POSITIVE
5,a well-made and often lovely depiction of the ...,POSITIVE
6,good movie .,POSITIVE
7,the best way to hope for any chance of enjoyin...,NEGATIVE
8,"some , like ballistic , arrive stillborn ... l...",NEGATIVE
9,reinforces the talents of screenwriter charlie...,POSITIVE


### 1.5 Visualise the explanations.
Unfortuantelly, HTML-heatmaps don't render correctly in GitHub preview ;(

In [8]:
labels = list(map(lambda i: "Predicted label: " + i, decode_labels(y_batch)))

In [9]:
# Visualise Integrated Gradients explanations.
a_batch_int_grad = map_explanations(
    qn.explain(model, x_batch[:4], y_batch[:4], method="IntGrad", batch_interpolated_inputs=True),
    qn.normalize_sum_to_1,
)

HTML(
    qn.visualise_explanations_as_html(
        a_batch_int_grad, labels=labels, ignore_special_tokens=True
)
)

In [10]:
# Visualise SHAP explanations.
a_batch_shap = map_explanations(
    qn.explain(model, x_batch[:4], y_batch[:4], method="SHAP", call_kwargs={"max_evals": 10}),
    qn.normalize_sum_to_1,
)
HTML(qn.visualise_explanations_as_html(a_batch_shap, labels=labels))

## 2) Quantitative analysis using Quantus.
To see all available metric and their category, we can run `quantus.nlp.available_metrics()`

In [15]:
qn.available_metrics()

{'Robustness': ['Max-Sensitivity',
  'Avg-Sensitivity',
  'Relative Input Stability',
  'Relative Output Stability',
  'Relative Representation Stability'],
 'Randomisation': ['Model Parameter Randomisation', 'Random Logit'],
 'Faithfulness': ['Token Flipping']}

In [11]:
# We will need it later
# fmt: off
unk_token_embedding = model.embedding_lookup([model.tokenizer._tokenizer.unk_token_id])[0, 0]  # noqa
# fmt: on

metrics = {
    # By default, perturbation is applied to plain-text inputs.
    "Average Sensitivity": qn.AvgSensitivity(nr_samples=10, disable_warnings=True),
    "Max Sensitivity": qn.MaxSensitivity(
        nr_samples=10,
        # Perturbation type is inferred from perturb_func signature.
        perturb_func=qn.gaussian_noise,
        disable_warnings=True,
    ),
    "Relative Input Stability": qn.RelativeInputStability(
        nr_samples=10,
        disable_warnings=True,
    ),
    # By default, we normalise scores, so they sum up to 1, this behaviour can be disabled
    # Additionally we can run evaluation on absolute values of explanation scores.
    "Relative Output Stability": qn.RelativeOutputStability(
        nr_samples=10,
        disable_warnings=True,
    ),
    "Relative Representation Stability": qn.RelativeRepresentationStability(
        nr_samples=10,
        disable_warnings=True,
    ),
    "Model Parameter Randomisation": qn.ModelParameterRandomisation(
        seed=42,
        disable_warnings=True,
    ),
    "Random Logit": qn.RandomLogit(num_classes=2, seed=42, disable_warnings=True),
    "Token Flipping": qn.TokenFlipping(disable_warnings=True, abs=True),
}

# By default, qn.explain is used to generate explanations.
call_kwargs = {
    # We use GradXInput as default method for all methods.
    "explain_func_kwargs": {"method": "GradXInput"},
    # We evaluate Relative Input Stability for IntGrad with different baselines.
    "Relative Input Stability": [
        {
            "explain_func_kwargs": {
                "method": "IntGrad",
                "batch_interpolated_inputs": True,
            }
        },
        {
            "explain_func_kwargs": {
                "method": "IntGrad",
                "baseline_fn": lambda x: unk_token_embedding,
                "batch_interpolated_inputs": True,
            }
        },
    ],
}

# Notice, that no y_batch is required and explain_func_kwargs are passed only to metrics' __call__ method.
result = qn.evaluate(metrics, model, x_batch, call_kwargs=call_kwargs, run_gc=False)
gc.collect()

Evaluation...:   0%|          | 0/7 [00:00<?, ?it/s]

The settings for perturbing input e.g., 'perturb_func' didn't cause change in input. Reconsider the parameter settings.


#### 2.2 Results visualisation

For pruning task (default one) the tokens are removed in order of increasing relevance scores.
We expect Mean Squared Error to increase as we remove more important ones, which act as a proof that higher scores are assigned to important features.

In [ ]:
qn.plot_token_flipping_experiment(
    result["Token Flipping"],
    model.predict(x_batch).argmax(axis=-1),
    task="prunning"
)

In [ ]:
ris = result["Relative Input Stability"]
pd.DataFrame(ris).T.plot().legend(["[0..0] as Baseline", "[UNK] as baseline"])
plt.yscale("log")
plt.title("Relative Input Stability")
plt.xticks([])

In [ ]:
mpr = result["Model Parameter Randomisation"]
plot_model_parameter_randomisation_experiment(mpr)

In [ ]:
pd.DataFrame.from_dict(
    {
        k: v
        for k, v in result.items()
        if k
        not in (
            "ModelParameterRandomisation",
            "Relative Input Stability",
            "Token Flipping",
        )
    }
).plot()
plt.yscale("log")
plt.title("Quantus NLP metrics comparison")